In [1]:
enable_wandb = True
if enable_wandb:
    import wandb

In [2]:
import os
import pdb
import torch
import pandas

os.environ['TORCH'] = torch.__version__
print(torch.__version__)

2.4.0+cu124


In [3]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

def visualize(h, color):
    z = TSNE(n_components=2).fit_transform(h.detach().cpu().numpy())
    plt.figure(figsize=(10,10))
    plt.xticks([])
    plt.yticks([])
    plt.scatter(z[:, 0], z[:, 1], s=70, c=color, cmap="Set2")
    plt.show()

def embedding_to_wandb(h, color, key="embedding"):
    num_components = h.shape[-1]
    df = pandas.DataFrame(data=h.detach().cpu().numpy(),
                        columns=[f"c_{i}" for i in range(num_components)])
    df["target"] = color.detach().cpu().numpy().astype("str")
    cols = df.columns.tolist()
    df = df[cols[-1:] + cols[:-1]]
    wandb.log({key: df})

In [4]:
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures

dataset = Planetoid(root='../datasets/Planetoid', name='Cora', transform=NormalizeFeatures())

print()
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object.

print()
print(data)
print('===========================================================================================================')

# Gather some statistics about the graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')


Dataset: Cora():
Number of graphs: 1
Number of features: 1433
Number of classes: 7

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])
Number of nodes: 2708
Number of edges: 10556
Average node degree: 3.90
Number of training nodes: 140
Training node label rate: 0.05
Has isolated nodes: False
Has self-loops: False
Is undirected: True


#### Train an MLP

In [5]:
import torch
from torch.nn import Linear
import torch.nn.functional as F

class MLP(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        torch.manual_seed(12345)
        self.lin1 = Linear(dataset.num_features, hidden_channels)
        self.lin2 = Linear(hidden_channels, dataset.num_classes)

    def forward(self, x):
        x = self.lin1(x)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x

model = MLP(hidden_channels=16)
print(model)

MLP(
  (lin1): Linear(in_features=1433, out_features=16, bias=True)
  (lin2): Linear(in_features=16, out_features=7, bias=True)
)


In [6]:
if enable_wandb:
    wandb.init(project='node-classification')
    summary = dict()
    summary["data"] = dict()
    summary["data"]["num_features"] = dataset.num_features
    summary["data"]["num_classes"] = dataset.num_classes
    summary["data"]["num_nodes"] = data.num_nodes
    summary["data"]["num_edges"] = data.num_edges 
    summary["data"]["has_isolated_nodes"] = data.has_isolated_nodes()
    summary["data"]["has_self_nodes"] = data.has_self_loops()
    summary["data"]["is_undirected"] = data.is_undirected()
    summary["data"]["num_training_nodes"] = data.train_mask.sum()
    wandb.summary = summary

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: deepakpokkalla. Use `wandb login --relogin` to force relogin


In [7]:
model = MLP(hidden_channels=16)

with torch.no_grad():
  out = model(data.x)

if enable_wandb:
    embedding_to_wandb(out, color=data.y, key="mlp/embedding/init")
else:
    visualize(out, data.y)

criterion = torch.nn.CrossEntropyLoss()  # Define loss criterion.
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)  # Define optimizer.

def train():
      model.train()
      optimizer.zero_grad()  # Clear gradients.
      out = model(data.x)  # Perform a single forward pass.
      loss = criterion(out[data.train_mask], data.y[data.train_mask])  # Compute the loss solely based on the training nodes.
      loss.backward()  # Derive gradients.
      optimizer.step()  # Update parameters based on gradients.
      return loss

def test():
      model.eval()
      out = model(data.x)
      pred = out.argmax(dim=1)  # Use the class with highest probability.
      test_correct = pred[data.test_mask] == data.y[data.test_mask]  # Check against ground-truth labels.
      test_acc = int(test_correct.sum()) / int(data.test_mask.sum())  # Derive ratio of correct predictions.
      return test_acc

for epoch in range(1, 201):
    loss = train()
    if enable_wandb:
        wandb.log({"mlp/loss": loss})
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

Epoch: 001, Loss: 1.9616
Epoch: 002, Loss: 1.9564
Epoch: 003, Loss: 1.9495
Epoch: 004, Loss: 1.9419
Epoch: 005, Loss: 1.9369
Epoch: 006, Loss: 1.9244
Epoch: 007, Loss: 1.9196
Epoch: 008, Loss: 1.9131
Epoch: 009, Loss: 1.9019
Epoch: 010, Loss: 1.8895
Epoch: 011, Loss: 1.8737
Epoch: 012, Loss: 1.8616
Epoch: 013, Loss: 1.8539
Epoch: 014, Loss: 1.8458
Epoch: 015, Loss: 1.8197
Epoch: 016, Loss: 1.8194
Epoch: 017, Loss: 1.8027
Epoch: 018, Loss: 1.7732
Epoch: 019, Loss: 1.7733
Epoch: 020, Loss: 1.7491
Epoch: 021, Loss: 1.7472
Epoch: 022, Loss: 1.7194
Epoch: 023, Loss: 1.6962
Epoch: 024, Loss: 1.6584
Epoch: 025, Loss: 1.6668
Epoch: 026, Loss: 1.6226
Epoch: 027, Loss: 1.6333
Epoch: 028, Loss: 1.6029
Epoch: 029, Loss: 1.5534
Epoch: 030, Loss: 1.5783
Epoch: 031, Loss: 1.5381
Epoch: 032, Loss: 1.5213
Epoch: 033, Loss: 1.4926
Epoch: 034, Loss: 1.4652
Epoch: 035, Loss: 1.4754
Epoch: 036, Loss: 1.4559
Epoch: 037, Loss: 1.3932
Epoch: 038, Loss: 1.3604
Epoch: 039, Loss: 1.3321
Epoch: 040, Loss: 1.3120


In [8]:
test_acc = test()

out = model(data.x)
if enable_wandb:
    embedding_to_wandb(out, color=data.y, key="mlp/embedding/trained")
    wandb.summary["mlp/accuracy"] = test_acc
    wandb.log({"mlp/accuracy": test_acc})
else:
  visualize(out, data.y)

print(f'Test Accuracy: {test_acc:.4f}')

Test Accuracy: 0.6040


#### Train a GNN

In [9]:
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(dataset.num_features,hidden_channels)
        self.conv2 = GCNConv(hidden_channels, dataset.num_classes)
    
    def forward(self,x,edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return x

model = GCN(hidden_channels=16)
print(model)

GCN(
  (conv1): GCNConv(1433, 16)
  (conv2): GCNConv(16, 7)
)


In [10]:
model = GCN(16)
model.eval()

out = model(data.x, data.edge_index)

if enable_wandb:
    embedding_to_wandb(out, color=data.y, key="gcn/embedding/init")
else:
    visualize(out, data.y)

In [11]:
model = GCN(hidden_channels=16)
if enable_wandb:
    wandb.watch(model)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

def train():
      model.train()
      optimizer.zero_grad()  # Clear gradients.
      out = model(data.x, data.edge_index)  # Perform a single forward pass.
      loss = criterion(out[data.train_mask], data.y[data.train_mask])  # Compute the loss solely based on the training nodes.
      loss.backward()  # Derive gradients.
      optimizer.step()  # Update parameters based on gradients.
      return loss

def test():
      model.eval()
      out = model(data.x, data.edge_index)
      pred = out.argmax(dim=1)  # Use the class with highest probability.
      test_correct = pred[data.test_mask] == data.y[data.test_mask]  # Check against ground-truth labels.
      test_acc = int(test_correct.sum()) / int(data.test_mask.sum())  # Derive ratio of correct predictions.
      return test_acc


for epoch in range(1, 101):
    loss = train()
    if enable_wandb:
        wandb.log({"gcn/loss": loss})
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

Epoch: 001, Loss: 1.9458
Epoch: 002, Loss: 1.9502
Epoch: 003, Loss: 1.9353
Epoch: 004, Loss: 1.9271
Epoch: 005, Loss: 1.9169
Epoch: 006, Loss: 1.9113
Epoch: 007, Loss: 1.9394
Epoch: 008, Loss: 1.9089
Epoch: 009, Loss: 1.8967
Epoch: 010, Loss: 1.8916
Epoch: 011, Loss: 1.8978
Epoch: 012, Loss: 1.8800
Epoch: 013, Loss: 1.8824
Epoch: 014, Loss: 1.8631
Epoch: 015, Loss: 1.8680
Epoch: 016, Loss: 1.8650
Epoch: 017, Loss: 1.8482
Epoch: 018, Loss: 1.8586
Epoch: 019, Loss: 1.8370
Epoch: 020, Loss: 1.8349
Epoch: 021, Loss: 1.8152
Epoch: 022, Loss: 1.8143
Epoch: 023, Loss: 1.8129
Epoch: 024, Loss: 1.8044
Epoch: 025, Loss: 1.8160
Epoch: 026, Loss: 1.7999
Epoch: 027, Loss: 1.7881
Epoch: 028, Loss: 1.8017
Epoch: 029, Loss: 1.7710
Epoch: 030, Loss: 1.7840
Epoch: 031, Loss: 1.7550
Epoch: 032, Loss: 1.7180
Epoch: 033, Loss: 1.7393
Epoch: 034, Loss: 1.8064
Epoch: 035, Loss: 1.7348
Epoch: 036, Loss: 1.7096
Epoch: 037, Loss: 1.6780
Epoch: 038, Loss: 1.6875
Epoch: 039, Loss: 1.6763
Epoch: 040, Loss: 1.6855


In [12]:
test_acc = test()
print(f'Test Accuracy: {test_acc:.4f}')

Test Accuracy: 0.7330


In [13]:
model.eval()

out = model(data.x, data.edge_index)

if enable_wandb:
    wandb.summary["gcn/accuracy"] = test_acc
    wandb.log({"gcn/accuracy": test_acc})
    embedding_to_wandb(out, color=data.y, key="gcn/embedding/trained")
    wandb.finish()
else:
    visualize(out, data.y)

gcn/accuracy,▁
gcn/loss,███▇▇▇▇▇▇▇▆▆▆▆▆▆▆▅▅▆▅▅▅▅▄▄▄▃▃▃▃▄▃▃▂▂▁▂▂▁
mlp/accuracy,▁
mlp/loss,█████▇▇▆▆▆▆▅▄▄▄▄▃▃▂▂▂▂▂▂▁▂▁▂▁▁▁▁▁▂▁▁▁▁▁▂
gcn/accuracy,0.733
gcn/loss,1.16221
mlp/accuracy,0.604
mlp/loss,0.46549


In [14]:
assert enable_wandb, "W&B not enabled. Please, enable W&B and restart the notebook"

In [15]:
import tqdm

def agent_fn():
    wandb.init()
    model = GCN(hidden_channels=wandb.config.hidden_channels)
    wandb.watch(model)

    with torch.no_grad():
      out = model(data.x, data.edge_index)
      embedding_to_wandb(out, color=data.y, key="gcn/embedding/init")

    optimizer = torch.optim.Adam(model.parameters(), lr=wandb.config.lr, weight_decay=wandb.config.weight_decay)
    criterion = torch.nn.CrossEntropyLoss()

    def train():
          model.train()
          optimizer.zero_grad()  # Clear gradients.
          out = model(data.x, data.edge_index)  # Perform a single forward pass.
          loss = criterion(out[data.train_mask], data.y[data.train_mask])  # Compute the loss solely based on the training nodes.
          loss.backward()  # Derive gradients.
          optimizer.step()  # Update parameters based on gradients.
          return loss

    def test():
          model.eval()
          out = model(data.x, data.edge_index)
          pred = out.argmax(dim=1)  # Use the class with highest probability.
          test_correct = pred[data.test_mask] == data.y[data.test_mask]  # Check against ground-truth labels.
          test_acc = int(test_correct.sum()) / int(data.test_mask.sum())  # Derive ratio of correct predictions.
          return test_acc


    for epoch in tqdm.tqdm(range(1, 101)):
        loss = train()
        wandb.log({"gcn/loss": loss})


    model.eval()

    out = model(data.x, data.edge_index)
    test_acc = test()
    wandb.summary["gcn/accuracy"] = test_acc
    wandb.log({"gcn/accuracy": test_acc})
    embedding_to_wandb(out, color=data.y, key="gcn/embedding/trained")
    wandb.finish()

In [16]:
sweep_config = {
    "name": "gcn-sweep",
    "method": "bayes",
    "metric": {
        "name": "gcn/accuracy",
        "goal": "maximize",
    },
    "parameters": {
        "hidden_channels": {
            "values": [8, 16, 32]
        },
        "weight_decay": {
            "distribution": "normal",
            "mu": 5e-4,
            "sigma": 1e-5,
        },
        "lr": {
            "min": 1e-4,
            "max": 1e-3
        }
    }
}

# Register the Sweep with W&B
sweep_id = wandb.sweep(sweep_config, project="node-classification")

Create sweep with ID: reuf9j38
Sweep URL: https://wandb.ai/deepakpokkalla/node-classification/sweeps/reuf9j38


In [17]:
# Run the Sweeps agent
wandb.agent(sweep_id, project="node-classification", function=agent_fn, count=50)

wandb: Agent Starting Run: jipt0p0l with config:
wandb: 	hidden_channels: 32
wandb: 	lr: 0.0005237706785240348
wandb: 	weight_decay: 0.0005099180254108767


100%|██████████| 100/100 [00:01<00:00, 66.06it/s]


gcn/accuracy,▁
gcn/loss,████▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▅▅▄▄▄▄▄▃▃▄▄▃▃▃▂▂▂▂▁
gcn/accuracy,0.656
gcn/loss,1.85847


wandb: Agent Starting Run: dc325fij with config:
wandb: 	hidden_channels: 32
wandb: 	lr: 0.000407479305099397
wandb: 	weight_decay: 0.0005065688541132341


100%|██████████| 100/100 [00:02<00:00, 40.55it/s]


...

100%|██████████| 100/100 [00:01<00:00, 63.06it/s]


gcn/accuracy,▁
gcn/loss,▆▄▅▅▅█▆▅▅▆▅▄▄▄▄▄▄▄▄▄▃▃▃▄▃▃▃▃▄▂▃▄▂▃▂▁▂▃▂▁
gcn/accuracy,0.613
gcn/loss,1.92079


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: ok85f0sg with config:
wandb: 	hidden_channels: 8
wandb: 	lr: 0.000487316690583025
wandb: 	weight_decay: 0.0005050406252171472


100%|██████████| 100/100 [00:01<00:00, 64.40it/s]


gcn/accuracy,▁
gcn/loss,█████▇▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▅▅▄▄▄▃▃▃▃▃▃▂▂▂▂▁▁
gcn/accuracy,0.567
gcn/loss,1.90945
